In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import astropy
from astropy.units import au, minute

In [16]:
# Local imports
import kepler_sieve
from asteroid_spline import spline_ast_data
from asteroid_direction import calc_dir_ast2obs
from db_utils import sp2df, df2db

In [14]:
# Column groups
cols_dir = ['ux', 'uy', 'uz']
cols_spline = cols_dir + ['LightTime']

In [3]:
# Get detection times
sp_name = 'KS.GetDetectionTimes'
params = dict()
dt = sp2df(sp_name=sp_name, params=params)

In [4]:
# Calculate range of times required for splines
pad = 16.0
mjd0 = np.floor(np.min(dt.mjd)) - pad
mjd1 = np.ceil(np.max(dt.mjd)) + pad

print(f'Date range for splines: {mjd0}-{mjd1}')

Date range for splines: 58254.0-59312.0


In [5]:
# Get block of asteroid detections (directions only)
sp_name = 'KS.GetDetectionDirections'
params = {
    'd0': 0,
    'd1': 100,
}
det = sp2df(sp_name=sp_name, params=params)

In [6]:
det

,DetectionID,mjd,ux,uy,uz
0,1,58270.167662,-0.940203,-0.205183,0.271880
1,2,58270.167662,-0.940593,-0.207164,0.269011
2,3,58270.167662,-0.942678,-0.201138,0.266274
3,4,58270.167662,-0.944177,-0.198029,0.263275
4,5,58270.167662,-0.944884,-0.197557,0.261085
...,...,...,...,...,...
94,95,58270.167662,-0.965812,-0.141616,0.217143
95,96,58270.167662,-0.964592,-0.142827,0.221728
96,97,58270.167662,-0.967082,-0.133162,0.216842
97,98,58270.167662,-0.966287,-0.137820,0.217475


In [22]:
# Extract array of detection times
t_det = det.mjd.values

In [23]:
# Shape of detection times
print(f't_det.shape = {t_det.shape}')

t_det.shape = (99,)


In [7]:
# Get asteroid directions in this time range
sp_name = 'KS.GetAsteroidDirections'
params = {
    'n0': 0,
    'n1': 100,
    'mjd0': mjd0,
    'mjd1': mjd1
}
ast_in = sp2df(sp_name=sp_name, params=params)

In [8]:
ast_in

,AsteroidID,TimeID,mjd,ux,uy,uz,LightTime
0,1,83888640,58256.0,-0.695441,0.695787,0.179561,21.312345
1,1,83894400,58260.0,-0.710843,0.680749,0.176872,21.715483
2,1,83900160,58264.0,-0.726426,0.664782,0.174268,22.114047
3,1,83905920,58268.0,-0.742106,0.647905,0.171747,22.507469
4,1,83911680,58272.0,-0.757805,0.630133,0.169305,22.895268
...,...,...,...,...,...,...,...
26230,99,85386240,59296.0,0.699529,-0.689512,-0.187704,23.987647
26231,99,85392000,59300.0,0.720561,-0.666549,-0.191059,23.757249
26232,99,85397760,59304.0,0.740492,-0.643310,-0.194485,23.517752
26233,99,85403520,59308.0,0.759320,-0.619866,-0.197986,23.269268


In [9]:
# Extract array of asteroid sample times
# t_det = ast_in.mjd.values

In [18]:
# Spline asteroid directions to match these times
ast = spline_ast_data(df_ast=ast_in, ts=t_det, cols_spline=cols_spline)

In [19]:
ast

,AsteroidID,mjd,ux,uy,uz,LightTime
0,1,58256.0,-0.695441,0.695787,0.179561,21.312345
1,1,58260.0,-0.710843,0.680749,0.176872,21.715483
2,1,58264.0,-0.726426,0.664782,0.174268,22.114047
3,1,58268.0,-0.742106,0.647905,0.171747,22.507469
4,1,58272.0,-0.757805,0.630133,0.169305,22.895268
...,...,...,...,...,...,...
2597260,99,59296.0,0.699529,-0.689512,-0.187704,23.987647
2597261,99,59300.0,0.720561,-0.666549,-0.191059,23.757249
2597262,99,59304.0,0.740492,-0.643310,-0.194485,23.517752
2597263,99,59308.0,0.759320,-0.619866,-0.197986,23.269268


In [18]:
# Extract vectors for directions
u_det = det[cols_dir].values
u_ast = ast[cols_dir].values


u_det.shape = (99, 3)
u_ast.shape = (26235, 3)


In [17]:
# Review shapes
print(f'u_det.shape = {u_det.shape}')
print(f'u_ast.shape = {u_ast.shape}')

(26235, 3)